In [ ]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
import socket
import numpy as np
import cv2
import pywt
import pydicom
import os
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import threading
from PIL import Image, ImageTk

class DicomSenderGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("Transmission d'Images DICOM Chiffrées")
        self.root.geometry("800x700")
        self.root.configure(bg='#f0f0f0')
        
        # Variables
        self.dicom_path = tk.StringVar()
        self.server_ip = tk.StringVar(value='192.168.96.2')
        self.server_port = tk.StringVar(value='5050')
        self.encryption_mode = tk.StringVar(value='CBC')
        self.dwt_wavelet = tk.StringVar(value='haar')
        self.dwt_level = tk.StringVar(value='1')
        self.quantization_step = tk.StringVar(value='10')
        
        self.setup_ui()
        
    def setup_ui(self):
        # Style configuration
        style = ttk.Style()
        style.theme_use('clam')
        
        # Main container avec scrollbar si nécessaire
        canvas = tk.Canvas(self.root, bg='#f0f0f0')
        scrollbar = ttk.Scrollbar(self.root, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)
        
        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
        )
        
        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)
        
        # Main container
        main_frame = ttk.Frame(scrollable_frame, padding="20")
        main_frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))
        
        # Title
        title_label = ttk.Label(main_frame, text="Transmission d'Images DICOM Chiffrées", 
                               font=('Arial', 16, 'bold'))
        title_label.grid(row=0, column=0, columnspan=3, pady=(0, 20))
        
        # File selection frame
        file_frame = ttk.LabelFrame(main_frame, text="Sélection du fichier", padding="10")
        file_frame.grid(row=1, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(0, 10))
        
        ttk.Entry(file_frame, textvariable=self.dicom_path, width=60).grid(row=0, column=0, padx=(0, 10))
        ttk.Button(file_frame, text="Parcourir", command=self.browse_file).grid(row=0, column=1)
        
        # Server configuration frame
        server_frame = ttk.LabelFrame(main_frame, text="Configuration du serveur", padding="10")
        server_frame.grid(row=2, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(0, 10))
        
        ttk.Label(server_frame, text="Adresse IP:").grid(row=0, column=0, sticky=tk.W, padx=(0, 5))
        ttk.Entry(server_frame, textvariable=self.server_ip, width=20).grid(row=0, column=1, padx=(0, 20))
        
        ttk.Label(server_frame, text="Port:").grid(row=0, column=2, sticky=tk.W, padx=(0, 5))
        ttk.Entry(server_frame, textvariable=self.server_port, width=10).grid(row=0, column=3)
        
        # Processing parameters frame
        params_frame = ttk.LabelFrame(main_frame, text="Paramètres de traitement", padding="10")
        params_frame.grid(row=3, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(0, 10))
        
        # DWT parameters
        ttk.Label(params_frame, text="Ondelette DWT:").grid(row=0, column=0, sticky=tk.W, padx=(0, 5))
        wavelet_combo = ttk.Combobox(params_frame, textvariable=self.dwt_wavelet, 
                                    values=['haar', 'db4', 'db8', 'bior2.2', 'coif2'], width=15)
        wavelet_combo.grid(row=0, column=1, padx=(0, 20))
        
        ttk.Label(params_frame, text="Niveau DWT:").grid(row=0, column=2, sticky=tk.W, padx=(0, 5))
        ttk.Spinbox(params_frame, from_=1, to=5, textvariable=self.dwt_level, width=10).grid(row=0, column=3)
        
        ttk.Label(params_frame, text="Pas de quantification:").grid(row=1, column=0, sticky=tk.W, padx=(0, 5), pady=(10, 0))
        ttk.Entry(params_frame, textvariable=self.quantization_step, width=15).grid(row=1, column=1, pady=(10, 0))
        
        # Encryption frame
        encrypt_frame = ttk.LabelFrame(main_frame, text="Configuration du chiffrement", padding="10")
        encrypt_frame.grid(row=4, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(0, 10))
        
        ttk.Label(encrypt_frame, text="Mode de chiffrement:").grid(row=0, column=0, sticky=tk.W, padx=(0, 10))
        mode_combo = ttk.Combobox(encrypt_frame, textvariable=self.encryption_mode,
                                 values=['CBC', 'CFB', 'OFB', 'CTR'], width=15)
        mode_combo.grid(row=0, column=1)
        
        # Preview frame
        preview_frame = ttk.LabelFrame(main_frame, text="Aperçu de l'image", padding="10")
        preview_frame.grid(row=5, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(0, 10))
        
        self.preview_label = ttk.Label(preview_frame, text="Aucune image sélectionnée")
        self.preview_label.grid(row=0, column=0)
        
        # Progress frame
        progress_frame = ttk.Frame(main_frame)
        progress_frame.grid(row=6, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(0, 10))
        
        self.progress = ttk.Progressbar(progress_frame, mode='indeterminate')
        self.progress.grid(row=0, column=0, sticky=(tk.W, tk.E), padx=(0, 10))
        
        self.status_label = ttk.Label(progress_frame, text="Prêt")
        self.status_label.grid(row=0, column=1)
        
        # Buttons frame
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.grid(row=7, column=0, columnspan=3, sticky=(tk.W, tk.E), pady=(10, 0))
        
        # Centrer les boutons
        buttons_frame.columnconfigure(0, weight=1)
        buttons_frame.columnconfigure(1, weight=0)
        buttons_frame.columnconfigure(2, weight=0)
        buttons_frame.columnconfigure(3, weight=0)
        buttons_frame.columnconfigure(4, weight=1)
        
        ttk.Button(buttons_frame, text="Prévisualiser", command=self.preview_image).grid(row=0, column=1, padx=5)
        ttk.Button(buttons_frame, text="Envoyer", command=self.send_image, 
                  style='Accent.TButton').grid(row=0, column=2, padx=5)
        ttk.Button(buttons_frame, text="Quitter", command=self.root.quit).grid(row=0, column=3, padx=5)
        
        # Configure grid weights
        main_frame.columnconfigure(0, weight=1)
        scrollable_frame.columnconfigure(0, weight=1)
        progress_frame.columnconfigure(0, weight=1)
        
        # Pack canvas and scrollbar
        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")
        
        # Configure root
        self.root.columnconfigure(0, weight=1)
        self.root.rowconfigure(0, weight=1)
        
    def browse_file(self):
        """Ouvre le dialogue de sélection de fichier"""
        filetypes = [
            ("Fichiers DICOM", "*.dcm"),
            ("Images", "*.png *.jpg *.jpeg *.bmp *.tiff"),
            ("Tous les fichiers", "*.*")
        ]
        filename = filedialog.askopenfilename(
            title="Sélectionner un fichier image",
            filetypes=filetypes
        )
        if filename:
            self.dicom_path.set(filename)
            self.preview_image()
    
    def load_dicom_image(self, path):
        """Charge une image DICOM ou standard"""
        try:
            ext = os.path.splitext(path)[-1].lower()
            if ext == ".dcm":
                dicom = pydicom.dcmread(path)
                image = dicom.pixel_array.astype(np.float32)
                # Normalisation améliorée
                if np.max(image) > np.min(image):
                    image = 255 * (image - np.min(image)) / (np.max(image) - np.min(image))
                image = image.astype(np.uint8)
            else:
                image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                if image is None:
                    raise ValueError("Impossible de charger l'image : " + path)
            return image
        except Exception as e:
            raise ValueError(f"Erreur lors du chargement de l'image : {str(e)}")
    
    def apply_dwt(self, image, wavelet='haar', level=1):
        """Applique la transformée en ondelettes discrète"""
        coeffs = pywt.wavedec2(image, wavelet=wavelet, level=level)
        coeff_array, _ = pywt.coeffs_to_array(coeffs)
        return coeff_array
    
    def quantize(self, coeff_array, step=10):
        """Quantifie les coefficients"""
        return np.round(coeff_array / step) * step
    
    def pad_image(self, image):
        """Ajoute du padding pour l'alignement AES"""
        h, w = image.shape
        pad_h = (16 - h % 16) if h % 16 != 0 else 0
        pad_w = (16 - w % 16) if w % 16 != 0 else 0
        padded = np.pad(image, ((0, pad_h), (0, pad_w)), mode='constant')
        return padded, (pad_h, pad_w)
    
    def encrypt_aes(self, image, key, mode_name):
        """Chiffre l'image avec AES"""
        padded, pad = self.pad_image(image)
        image_bytes = padded.tobytes()
        
        mode_dict = {
            "CBC": AES.MODE_CBC,
            "CFB": AES.MODE_CFB,
            "OFB": AES.MODE_OFB,
            "CTR": AES.MODE_CTR
        }
        mode = mode_dict[mode_name]
        
        if mode_name == "CTR":
            cipher = AES.new(key, mode)
            ciphertext = cipher.encrypt(image_bytes)
            return ciphertext, cipher.nonce, pad
        else:
            iv = get_random_bytes(16)
            cipher = AES.new(key, mode, iv=iv)
            ciphertext = cipher.encrypt(image_bytes)
            return ciphertext, iv, pad
    
    def preview_image(self):
        """Affiche un aperçu de l'image sélectionnée"""
        try:
            if not self.dicom_path.get():
                self.status_label.config(text="Aucun fichier sélectionné")
                return
                
            self.status_label.config(text="Chargement de l'aperçu...")
            self.root.update_idletasks()
            
            image = self.load_dicom_image(self.dicom_path.get())
            
            # Redimensionner pour l'aperçu
            h, w = image.shape
            max_size = 200
            if h > w:
                new_h = max_size
                new_w = int(w * max_size / h)
            else:
                new_w = max_size
                new_h = int(h * max_size / w)
            
            resized = cv2.resize(image, (new_w, new_h))
            
            # Convertir pour Tkinter
            pil_image = Image.fromarray(resized)
            photo = ImageTk.PhotoImage(pil_image)
            
            self.preview_label.config(image=photo, text="")
            self.preview_label.image = photo  # Garder une référence
            
            self.status_label.config(text=f"Image chargée: {w}x{h} pixels")
            
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors du chargement de l'image:\n{str(e)}")
            self.status_label.config(text="Erreur de chargement")
            self.preview_label.config(image='', text="Erreur de chargement")
    
    def validate_inputs(self):
        """Valide les entrées utilisateur"""
        if not self.dicom_path.get():
            messagebox.showerror("Erreur", "Veuillez sélectionner un fichier image.")
            return False
        
        if not os.path.exists(self.dicom_path.get()):
            messagebox.showerror("Erreur", "Le fichier sélectionné n'existe pas.")
            return False
        
        try:
            port = int(self.server_port.get())
            if port < 1 or port > 65535:
                raise ValueError("Port hors limites")
        except ValueError:
            messagebox.showerror("Erreur", "Le port doit être un nombre entier entre 1 et 65535.")
            return False
        
        try:
            float(self.quantization_step.get())
            level = int(self.dwt_level.get())
            if level < 1 or level > 5:
                raise ValueError("Niveau DWT hors limites")
        except ValueError:
            messagebox.showerror("Erreur", "Les paramètres numériques doivent être des nombres valides.")
            return False
        
        return True
    
    def send_image_thread(self):
        """Thread pour l'envoi de l'image"""
        try:
            self.root.after(0, lambda: self.status_label.config(text="Chargement de l'image..."))
            image = self.load_dicom_image(self.dicom_path.get())
            
            self.root.after(0, lambda: self.status_label.config(text="Transformation DWT..."))
            dwt_array = self.apply_dwt(image, 
                                     wavelet=self.dwt_wavelet.get(), 
                                     level=int(self.dwt_level.get()))
            
            self.root.after(0, lambda: self.status_label.config(text="Quantification..."))
            quantized = self.quantize(dwt_array.copy(), 
                                    step=float(self.quantization_step.get())).astype(np.uint8)
            
            self.root.after(0, lambda: self.status_label.config(text="Chiffrement..."))
            key = b'MasterSIDI2025__'
            ciphertext, iv_or_nonce, pad = self.encrypt_aes(quantized, key, self.encryption_mode.get())
            
            self.root.after(0, lambda: self.status_label.config(text="Connexion au serveur..."))
            with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
                s.settimeout(10)  # Timeout de 10 secondes
                s.connect((self.server_ip.get(), int(self.server_port.get())))
                
                self.root.after(0, lambda: self.status_label.config(text="Envoi des métadonnées..."))
                shape = quantized.shape
                meta = f"{shape[0]},{shape[1]},{pad[0]},{pad[1]},{self.encryption_mode.get()},{iv_or_nonce.hex()}"
                meta_bytes = meta.encode()
                s.sendall(len(meta_bytes).to_bytes(4, 'big'))
                s.sendall(meta_bytes)
                
                self.root.after(0, lambda: self.status_label.config(text="Envoi de l'image chiffrée..."))
                s.sendall(len(ciphertext).to_bytes(4, 'big'))
                s.sendall(ciphertext)
            
            self.root.after(0, lambda: self.status_label.config(text="Image envoyée avec succès !"))
            self.root.after(0, lambda: messagebox.showinfo("Succès", "Image envoyée avec succès au serveur !"))
            
        except socket.timeout:
            self.root.after(0, lambda: self.status_label.config(text="Erreur: Timeout de connexion"))
            self.root.after(0, lambda: messagebox.showerror("Erreur", "Timeout de connexion au serveur."))
        except ConnectionRefusedError:
            self.root.after(0, lambda: self.status_label.config(text="Erreur: Connexion refusée"))
            self.root.after(0, lambda: messagebox.showerror("Erreur", "Connexion refusée par le serveur."))
        except Exception as e:
            self.root.after(0, lambda: self.status_label.config(text="Erreur d'envoi"))
            self.root.after(0, lambda: messagebox.showerror("Erreur", f"Erreur lors de l'envoi:\n{str(e)}"))
        finally:
            self.root.after(0, self.progress.stop)
    
    def send_image(self):
        """Lance l'envoi de l'image dans un thread séparé"""
        if not self.validate_inputs():
            return
        
        self.progress.start()
        thread = threading.Thread(target=self.send_image_thread, daemon=True)
        thread.start()

def main():
    root = tk.Tk()
    app = DicomSenderGUI(root)
    root.mainloop()

if __name__ == "__main__":
    main()